## Load environment and check for folder

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import subprocess
import os
base_dir = "/content/gdrive/MyDrive/final-project-paralelizacao-tecnica-de-meios-tons"
get_ipython().run_line_magic('cd', base_dir)

# assert that we have the exercise folder
assert 'repositorio' in os.listdir(), "Exercise folder not found"

In [ ]:
import os
import subprocess
import time
from datetime import datetime
from PIL import Image

# Configuração de diretórios
base_dir = os.getcwd()
exercise_dir = os.path.join(base_dir, "repositorio")
build_dir = os.path.join(exercise_dir, "build")
input_base_dir = os.path.join(exercise_dir, "img", "ppm")
output_base_dir = os.path.join(exercise_dir, "out")
log_dir = os.path.join(exercise_dir, "logs")

# Encontrar todas as imagens ppm no diretório de entrada
test_images = []
for root, dirs, files in os.walk(input_base_dir):
    for file in files:
        if file.endswith(".ppm"):
            full_path = os.path.join(root, file)
            rel_path = os.path.relpath(full_path, input_base_dir)
            test_images.append(rel_path)

Cuda de Técnicas de de Meios-Tons

In [ ]:
import os
import subprocess
import time
from datetime import datetime
from PIL import Image

# Configuração de diretórios
base_dir = os.getcwd()
exercise_dir = os.path.join(base_dir, "repositorio")
build_dir = os.path.join(exercise_dir, "build")
input_base_dir = os.path.join(exercise_dir, "img", "ppm")
output_base_dir = os.path.join(exercise_dir, "out")
log_dir = os.path.join(exercise_dir, "logs")

# Criar diretórios necessários
os.makedirs(log_dir, exist_ok=True)
os.makedirs(output_base_dir, exist_ok=True)

# Arquivos de log
log_filename = f"execution_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
log_filepath = os.path.join(log_dir, log_filename)
times_filename = f"execution_times_by_size_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
times_filepath = os.path.join(log_dir, times_filename)

# Função para obter dimensões da imagem
def get_image_dimensions(image_path):
    try:
        with Image.open(image_path) as img:
            return img.size  # retorna (width, height)
    except Exception as e:
        print(f"Erro ao obter dimensões de {image_path}: {str(e)}")
        return (0, 0)

# Função para registrar logs
def log_message(message, print_to_console=True):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_entry = f"[{timestamp}] {message}\n"
    
    with open(log_filepath, 'a') as log_file:
        log_file.write(log_entry)
    
    if print_to_console:
        print(message)

# Função para registrar tempos por tamanho
def log_time_by_size(image_path, dimensions, execution_time):
    size_str = f"{dimensions[0]}x{dimensions[1]}"
    entry = f"{size_str},{dimensions[0]},{dimensions[1]},{execution_time:.4f}\n"
    
    with open(times_filepath, 'a') as times_file:
        times_file.write(entry)

# Inicializar arquivos de log
with open(log_filepath, 'w') as log_file:
    log_file.write("Log de Execução - Processamento de Imagens\n")
    log_file.write(f"Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

with open(times_filepath, 'w') as times_file:
    times_file.write("Tamanho,Largura,Altura,Tempo(s)\n")

# Encontrar todas as imagens ppm no diretório de entrada
test_images = []
for root, dirs, files in os.walk(input_base_dir):
    for file in files:
        if file.endswith(".ppm"):
            full_path = os.path.join(root, file)
            rel_path = os.path.relpath(full_path, input_base_dir)
            test_images.append(rel_path)

log_message(f"Total de imagens encontradas: {len(test_images)}")

# Configurações do processamento
exec_path = os.path.join(build_dir, "fsh_cuda")
subprocess.run(f"chmod +x {exec_path}", shell=True)
p = 0.5          # parâmetro de ruído
stochastic = 1    # usar dithering estocástico
grayscale_flag = "-g"  # converter para grayscale

# Processar imagens
for i, rel_image_path in enumerate(test_images, start=1):
    start_time = time.time()
    img_path = os.path.join(input_base_dir, rel_image_path)
    
    # Obter dimensões da imagem
    dimensions = get_image_dimensions(img_path)
    size_str = f"{dimensions[0]}x{dimensions[1]}"
    
    log_message(f"Processando imagem {i}/{len(test_images)}: {rel_image_path} ({size_str})")
    
    # Preparar caminho de saída
    output_path = os.path.join(output_base_dir, rel_image_path)
    output_dir = os.path.dirname(output_path)
    os.makedirs(output_dir, exist_ok=True)

    # Nome do arquivo de saída
    output_path = output_path.replace(".ppm", "_stochastic.ppm" if stochastic else ".ppm")

    # Executar comando
    command = f"{exec_path} {img_path} {output_path} {p} {stochastic} {grayscale_flag}"
    result = subprocess.run(
        command, shell=True, cwd=exercise_dir,
        stdout=subprocess.PIPE, stderr=subprocess.PIPE,
        text=True
    )

    # Calcular tempo de execução
    execution_time = time.time() - start_time
    
    # Registrar no log geral
    with open(log_filepath, 'a') as log_file:
        log_file.write(f"{rel_image_path};{size_str};{execution_time:.4f}\n")

    # Registrar no arquivo de tempos por tamanho
    log_time_by_size(rel_image_path, dimensions, execution_time)

    log_message(f"Concluído em {execution_time:.2f}s - Tamanho: {size_str}")

log_message(f"\nProcessamento concluído. Logs salvos em:")
log_message(f"- Log completo: {log_filepath}")
log_message(f"- Tempos por tamanho: {times_filepath}")